# 2D Nuclear Segmentation with Mask-RCNN

In [3]:
# This notebook is to benchmark the model output from the RetinaMask model trained on TissueNet
import os
import errno

import numpy as np

import deepcell

In [4]:
# create folder for this set of experiments
experiment_folder = "retina_mask/"
MODEL_DIR = os.path.join("/data/analyses/", experiment_folder)
NPZ_DIR = "/data/npz_data/20201018_freeze/"
LOG_DIR = '/data/logs'

if not os.path.isdir(MODEL_DIR):
    os.makedirs(MODEL_DIR)

In [6]:
def calc_jaccard_index_object(metric_predictions, true_labels, pred_labels):
    jacc_list = []
    for i in range(true_labels.shape[0]):
        y_true = true_labels[i, :, :, 0]
        y_pred = pred_labels[i, :, :, 0]
        true_ids = metric_predictions[i][0]['correct']['y_true']
        pred_ids = metric_predictions[i][0]['correct']['y_pred']

        current_accum = []

        for id in range(len(true_ids)):
            true_mask = y_true == true_ids[id]
            pred_mask = y_pred == pred_ids[id]

            current_jacc = (np.sum(np.logical_and(true_mask, pred_mask)) /
                np.sum(np.logical_or(true_mask, pred_mask)))
            current_accum.append(current_jacc)

        jacc_list.append(current_accum)
    return jacc_list


In [10]:
from tensorflow.keras.optimizers import SGD, Adam
from deepcell.utils.train_utils import rate_scheduler
from deepcell.utils.retinanet_anchor_utils import get_anchor_parameters
from deepcell.training import train_model_retinanet
from deepcell import model_zoo
from skimage.measure import label
from deepcell_toolbox.multiplex_utils import multiplex_preprocess
from deepcell_toolbox.retinanet import retinamask_postprocess
from timeit import default_timer

model_splits = ['1', '2', '3']
metrics = {}
for split in model_splits:
    print('loading data')
    test_name = "20201018_multiplex_seed_{}_test_256x256.npz".format(split)
    test_dict = np.load(NPZ_DIR + test_name)
    train_name = "20201018_multiplex_seed_{}_train_256x256.npz".format(split)
    train_dict = np.load(NPZ_DIR + train_name)
    y_train = train_dict['y']
    
    X_test = test_dict['X']
    X_test = multiplex_preprocess(X_test)
    
    y_test = test_dict['y']

    
    model_name = 'retina_mask_split_{}.h5'.format(split)
    backbone = 'resnet50'  # vgg16, vgg19, resnet50, densenet121, densenet169, densenet201

    # start timing
    time_start = default_timer()
    print('creating backbone levels')
    # Generate backbone information from the data
    backbone_levels, pyramid_levels, anchor_params = get_anchor_parameters(y_train.astype('int'))
    
    print('creating model')
    model = model_zoo.RetinaMask(
        backbone=backbone,
        input_shape=[256, 256, 2],
        class_specific_filter=False,
        num_classes=1,
        backbone_levels=backbone_levels,
        pyramid_levels=pyramid_levels,
        anchor_params=anchor_params
    )
    
    model.load_weights(MODEL_DIR + model_name)
    print('predicting')
    boxes, scores, labels, masks = model.predict(X_test)[-4:]
    print('postprocessing')
    masks =  retinamask_postprocess((boxes, scores, labels, masks))
    masks = np.expand_dims(masks, axis=-1)
    
    # end time
    time_end = default_timer()
    print("elapsed time is {}".format(time_end - time_start))
    
    
    for i in range(masks.shape[0]):
        img = masks[i, :, :, 0]
        img = label(img)
        masks[i, :, :, 0] = img
    
    for i in range(y_test.shape[0]):
        img = y_test[i, :, :, 0]
        img = label(img)
        y_test[i, :, :, 0] = img
        
    # calculating accuracy
    print("calculating accuracy")
    db = DatasetBenchmarker(y_true=y_test, 
                       y_pred=masks,
                       tissue_list=test_dict['tissue_list'],
                       platform_list=test_dict['platform_list'],
                       model_name='default_model')
    tissue_stats, platform_stats = db.benchmark()
    
    jacc = calc_jaccard_index_object(db.metrics.predictions, y_test, masks)
    jacc = np.concatenate(jacc)
    jacc_mean = np.mean(jacc)
    print(jacc_mean)
    metrics[split] = {'tissue_stats':tissue_stats, 'platform_stats': platform_stats, 'jacc': jacc_mean}

loading data
creating backbone levels
creating model
predicting
postprocessing
elapsed time is 477.0795951390028
calculating accuracy

____________Object-based statistics____________

Number of true cells:		 139873
Number of predicted cells:	 106624

Correct detections:  84286	Recall: 60.2589%
Incorrect detections: 22338	Precision: 79.0497%

Gained detections: 13527	Perc Error: 20.2907%
Missed detections: 48595	Perc Error: 72.8932%
Merges: 1581		Perc Error: 2.3715%
Splits: 2358		Perc Error: 3.537%
Catastrophes: 605		Perc Error: 0.9075%

Gained detections from splits: 3233
Missed detections from merges: 1712
True detections involved in catastrophes: 826
Predicted detections involved in catastrophes: 984 

Average Pixel IOU (Jaccard Index): 0.7033 

uid is breast
uid is gi
uid is immune
uid is lung
uid is pancreas
uid is skin
uid is codex
uid is cycif
uid is imc
uid is mibi
uid is mxif
uid is vectra
uid is all
0.7982397763231487
loading data
creating backbone levels
creating model
predic

/usr/local/lib/python3.6/dist-packages/deepcell_toolbox/retinanet.py:295: UserWarning: Random walker only segments unlabeled areas, where labels == 0. No zero valued areas in labels were found. Returning provided labels.
  segments = random_walker(foreground, markers)


elapsed time is 493.7807670969996
calculating accuracy

____________Object-based statistics____________

Number of true cells:		 146194
Number of predicted cells:	 107483

Correct detections:  86917	Recall: 59.4532%
Incorrect detections: 20566	Precision: 80.8658%

Gained detections: 12532	Perc Error: 18.1542%
Missed detections: 52046	Perc Error: 75.3951%
Merges: 1885		Perc Error: 2.7307%
Splits: 1993		Perc Error: 2.8871%
Catastrophes: 575		Perc Error: 0.833%

Gained detections from splits: 2649
Missed detections from merges: 2070
True detections involved in catastrophes: 810
Predicted detections involved in catastrophes: 965 

Average Pixel IOU (Jaccard Index): 0.7019 

uid is breast
uid is gi
uid is immune
uid is lung
uid is pancreas
uid is skin
uid is codex
uid is cycif
uid is imc
uid is mibi
uid is mxif
uid is vectra
uid is all
0.8046485243082276
loading data
creating backbone levels
creating model
predicting
postprocessing
elapsed time is 463.3777252949985
calculating accuracy

___

In [13]:
metrics['3']['jacc']

0.798053242914878

In [14]:
np.savez_compressed(os.path.join(MODEL_DIR, 'retinamask_metrics_jacc.npz'), **metrics)

In [5]:
# Copyright 2016-2020 The Van Valen Lab at the California Institute of
# Technology (Caltech), with support from the Paul Allen Family Foundation,
# Google, & National Institutes of Health (NIH) under Grant U24CA224309-01.
# All rights reserved.
#
# Licensed under a modified Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.github.com/vanvalenlab/caliban-toolbox/LICENSE
#
# The Work provided may be used for non-commercial academic purposes only.
# For any other use of the Work, including commercial use, please contact:
# vanvalenlab@gmail.com
#
# Neither the name of Caltech nor the names of its contributors may be used
# to endorse or promote products derived from this software without specific
# prior written permission.
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
import numpy as np

from deepcell_toolbox.metrics import Metrics, stats_pixelbased
from scipy.stats import hmean


class DatasetBenchmarker(object):
    """Class to perform benchmarking across different tissue and platform types

    Args:
        y_true: true labels
        y_pred: predicted labels
        tissue_list: list of tissue names for each image
        platform_list: list of platform names for each image
        model_name: name of the model used to generate the predictions
        metrics_kwargs: arguments to be passed to metrics package

    Raises:
        ValueError: if y_true and y_pred have different shapes
        ValueError: if y_true and y_pred are not 4D
        ValueError: if tissue_ids or platform_ids is not same length as labels
    """
    def __init__(self,
                 y_true,
                 y_pred,
                 tissue_list,
                 platform_list,
                 model_name,
                 metrics_kwargs={}):
        if y_true.shape != y_pred.shape:
            raise ValueError('Shape mismatch: y_true has shape {}, '
                             'y_pred has shape {}. Labels must have the same'
                             'shape.'.format(y_true.shape, y_pred.shape))
        if len(y_true.shape) != 4:
            raise ValueError('Data must be 4D, supplied data is {}'.format(y_true.shape))

        self.y_true = y_true
        self.y_pred = y_pred

        if len({y_true.shape[0], len(tissue_list), len(platform_list)}) != 1:
            raise ValueError('Tissue_list and platform_list must have same length as labels')

        self.tissue_list = tissue_list
        self.platform_list = platform_list
        self.model_name = model_name
        self.metrics = Metrics(model_name, **metrics_kwargs)

    def _benchmark_category(self, category_ids):
        """Compute benchmark stats over the different categories in supplied list

        Args:
            category_ids: list specifying which category each image belongs to

        Returns:
            stats_dict: dictionary of benchmarking results
        """

        unique_ids = np.unique(category_ids)

        # create dict to hold stats across each category
        stats_dict = {}
        for uid in unique_ids:
            print("uid is {}".format(uid))
            stats_dict[uid] = {}
            category_idx = np.isin(category_ids, uid)

            # sum metrics across individual images
            for key in self.metrics.stats:
                stats_dict[uid][key] = self.metrics.stats[key][category_idx].sum()

            # compute additional metrics not produced by Metrics class
            stats_dict[uid]['recall'] = \
                stats_dict[uid]['correct_detections'] / stats_dict[uid]['n_true']

            stats_dict[uid]['precision'] = \
                stats_dict[uid]['correct_detections'] / stats_dict[uid]['n_pred']

            stats_dict[uid]['f1'] = \
                hmean([stats_dict[uid]['recall'], stats_dict[uid]['precision']])

            pixel_stats = stats_pixelbased(self.y_true[category_idx] != 0,
                                           self.y_pred[category_idx] != 0)
            stats_dict[uid]['jaccard'] = pixel_stats['jaccard']

        return stats_dict

    def benchmark(self):
        self.metrics.calc_object_stats(self.y_true, self.y_pred)
        tissue_stats = self._benchmark_category(category_ids=self.tissue_list)
        platform_stats = self._benchmark_category(category_ids=self.platform_list)
        all_stats = self._benchmark_category(category_ids=['all'] * len(self.tissue_list))
        tissue_stats['all'] = all_stats['all']
        platform_stats['all'] = all_stats['all']

        return tissue_stats, platform_stats
